In [4]:
import pandas as pd
import numpy as np

In [5]:
data = pd.read_csv('19b7b17c-e655-4013-b4d1-7daa9727225f_NewYorkCityPropertySales.csv')

In [6]:
data.head()

,borough_code,borough_code_definition,neighborhood,building_class_category_code,building_class_category_code_definition,tax_class_at_present_code,tax_class_at_present_code_definition,building_class_at_present_code,building_class_at_present_code_definition,address,...,exempt_value_land,exempt_value_total,year_altered_one,year_altered_two,historical_district_name,maximum_allowable_residential_far,maximum_allowable_commercial_far,maximum_allowable_facility_far,x_coordinate,y_coordinate
0,1,Manhattan,ALPHABET CITY,07,RENTALS - WALKUP APARTMENTS,2,Includes all other property that is primarily ...,C4,OLD LAW TENEMENT,738 EAST 6 STREET,...,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,990363.0,202588.0
1,1,Manhattan,ALPHABET CITY,07,RENTALS - WALKUP APARTMENTS,2,Includes all other property that is primarily ...,C7,WALK-UP APT. OVER SIX FAMILIES WITH STORES,27 AVENUE C,...,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,989508.0,202158.0
2,1,Manhattan,ALPHABET CITY,07,RENTALS - WALKUP APARTMENTS,2A,Includes all other property that is primarily ...,C2,FIVE TO SIX FAMILIES,151 AVENUE B,...,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,989831.0,203960.0
3,1,Manhattan,ALPHABET CITY,07,RENTALS - WALKUP APARTMENTS,2A,Includes all other property that is primarily ...,C2,FIVE TO SIX FAMILIES,153 AVENUE B,...,NaN,NaN,1984.0,NaN,NaN,4.0,NaN,4.0,989833.0,203986.0
4,1,Manhattan,ALPHABET CITY,07,RENTALS - WALKUP APARTMENTS,2,Includes all other property that is primarily ...,C7,WALK-UP APT. OVER SIX FAMILIES WITH STORES,193 EAST 4 STREET,...,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.0,988623.0,203137.0


In [ ]:
def drop_rows_where_col_is_nan(df, col_name):
    df = df[pd.notnull(df[col_name])]
    return(df)

In [ ]:
def drop_definition(df):
    keyword = 'definition';
    cols = [c for c in df.columns if c[-len(keyword):] != keyword];
    return(df[cols])

In [ ]:
def replace_nan_by_zero(df, col_name):
    df[col_name] = df[col_name].fillna(0)
    return(df)

In [ ]:
def replace_nan_by_mean(df, col_name):
    df[col_name] = df[col_name].fillna(df[col_name].mean())
    return(df)

In [ ]:
def ampute_with_mean_and_add_extra_col(df, col_name):
    for x in col_name:
        df[x+'_amputed'] = df[x].isnull();
        df[x] = df[x].fillna(df[x].mean())
    return(df)

In [ ]:
def get_year(string):
    return(string[:4])

In [ ]:
def extract_month(string):
    return(string[5:7])

In [ ]:
def preprocess_function(df, fill_zero_cols=None, drop_cols=None, 
                        fill_mean_cols = None, fill_mean_add_col = None):
    df = drop_rows_where_col_is_nan(df, 'sale_price')
    df = drop_definition(df);
    if(fill_zero_cols is not None):
        df = replace_nan_by_zero(df, fill_zero_cols);
    if(drop_cols is not None):
        keep_cols = [c for c in df.columns if c not in drop_cols]
        df = df[keep_cols];
    if(fill_mean_cols is not None):
        df = replace_nan_by_mean(df, fill_mean_cols);
    if(fill_mean_add_col is not None):
        df = ampute_with_mean_and_add_extra_col(df, fill_mean_add_col);
        
    year = df['sale_date'].apply(get_year);
    df['sale_date'] = df['sale_date'].apply(extract_month);
    df['sale_year'] = (year == '2018')
    return(df)

In [ ]:
other_cols_to_be_dropped = ['borough_code', 'neighborhood', 'address', 'apartment_number', 
                            'total_units', 'x_coordinate', 'y_coordinate', 'owner_type', 'units_total_number_of',
                           'historical_district_name']

In [ ]:
fill_zero_cols = ['residential_units', 'commercial_units', 'buildings_number_of', 'floors_number_of',
       'units_residential_number_of', 'floor_area_total_building',
       'floor_area_commercial', 'floor_area_residential', 'floor_area_office',
       'floor_area_retail', 'floor_area_garage', 'floor_area_storage',
       'floor_area_factory', 'floor_area_other', 'assessed_value_land', 'assessed_value_total',
       'exempt_value_land', 'exempt_value_total', 'year_altered_one',
       'year_altered_two', 'school_district', 'city_council_district',
       'fire_company', 'police_precinct', 'health_center_district',
                 'tax_class_at_present_code', 'building_class_at_present_code', 'year_built']

In [ ]:
fill_mean_cols = ['maximum_allowable_residential_far', 'maximum_allowable_commercial_far', 
                'maximum_allowable_facility_far']

In [ ]:
fill_mean_add_extra_col = ['land_square_feet', 'gross_square_feet']

In [ ]:
df_processed = preprocess_function(data, drop_cols=other_cols_to_be_dropped, 
                                   fill_zero_cols=fill_zero_cols, fill_mean_cols=fill_mean_cols,
                                  fill_mean_add_col = fill_mean_add_extra_col)

In [ ]:
df_processed.isnull().sum()/df_processed.shape[0]

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()

In [ ]:
df_processed = df_processed.set_index('sale_price').reset_index()

In [ ]:
char_cols = df_processed.dtypes.pipe(lambda x: x[x == 'object']).index
label_mapping = {}

for c in char_cols:
    df_processed[c], label_mapping[c] = pd.factorize(df_processed[c])

In [ ]:
clf.fit(df_processed.iloc[:, 1:], df_processed.iloc[:, 0])